In [ ]:
!pip install rouge
!pip install bert-score


In [ ]:
import pandas as pd
import json
from rouge import Rouge
from bert_score import score
from tqdm import tqdm

# ===== File paths (Google Drive) =====
base_path = "/content/drive/MyDrive/csvs/"
ref_df = pd.read_csv(base_path + "DrawBenchPrompts.csv")
flux_df = pd.read_csv(base_path + "meta_captions_Flux-Dev.csv")
sd2_df = pd.read_csv(base_path + "meta_captions_sd_2.csv")
sdxl_df = pd.read_csv(base_path + "meta_captions_sdxl.csv")

# ===== Merge on image_name =====
merged = ref_df[["image_name", "Prompts", "Category"]].merge(
    flux_df[["image_name", "Meta Caption"]].rename(columns={"Meta Caption": "Flux-Dev"}),
    on="image_name"
).merge(
    sd2_df[["image_name", "Meta Caption"]].rename(columns={"Meta Caption": "sd_2"}),
    on="image_name"
).merge(
    sdxl_df[["image_name", "Meta Caption"]].rename(columns={"Meta Caption": "sdxl"}),
    on="image_name"
)

# ===== Metric Setup =====
rouge = Rouge()

# ===== Initialize accumulators =====
avg_scores = {
    "Flux-Dev": {"ROUGE-L_Recall": 0, "BERTScore_Recall": 0},
    "sd_2": {"ROUGE-L_Recall": 0, "BERTScore_Recall": 0},
    "sdxl": {"ROUGE-L_Recall": 0, "BERTScore_Recall": 0}
}

num_images = len(merged)

# ===== Compute ROUGE Recall + BERT Recall =====
for model in ["Flux-Dev", "sd_2", "sdxl"]:
    cands = merged[model].fillna("").astype(str).tolist()
    refs = merged["Prompts"].fillna("").astype(str).tolist()

    # --- ROUGE-L Recall ---
    rouge_recall_total = 0
    for ref, cand in zip(refs, cands):
        try:
            rouge_scores = rouge.get_scores(cand, ref)[0]
            rouge_recall_total += rouge_scores["rouge-l"]["r"]  # recall
        except:
            rouge_recall_total += 0.0
    avg_rouge_recall = rouge_recall_total / num_images

    # --- BERT Recall ---
    P, R, F1 = score(cands, refs, lang="en", verbose=True)
    avg_bert_recall = R.mean().item()

    # Save averages
    avg_scores[model]["ROUGE-L_Recall"] = avg_rouge_recall
    avg_scores[model]["BERTScore_Recall"] = avg_bert_recall

# ===== Save to JSON =====
out_path = base_path + "rouge,bert_recall_scores.json"
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(avg_scores, f, indent=4, ensure_ascii=False)

print(f" Recall scores saved to {out_path}")
